<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Summary_Generator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/dmmiller612/lecture-summarizer.git
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)

Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 507 (delta 100), reused 100 (delta 87), pack-reused 378
Receiving objects: 100% (507/507), 30.39 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [ ]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
all_files = os.listdir("/content/data/")   
contents = []
for file_name in tqdm(all_files):
  dir_name = "/content/data/" + file_name
  with open(dir_name) as f:
    content = f.read()
    contents.append(content)

df = pd.DataFrame()

df['content'] = contents
df.head()

100%|██████████| 13/13 [00:00<00:00, 8469.39it/s]


,content
0,Welcome back to Pattern Recognition!\nToday we...
1,Welcome back to Pattern Recognition!\nToday we...
2,Welcome back to pattern recognition.\nToday we...
3,Welcome back to pattern recognition.\nToday we...
4,"The Chrysler Building, the famous art deco New..."


In [ ]:
import textwrap

def generate_summary(pipe, paragraph):
  arr = textwrap.wrap(paragraph, 4000)
  summary = ''
  for i in arr:
    summary += pipe( i ) [0]["summary_text"].replace(" .", ".\n")
  return summary

In [ ]:
%pip install -U transformers

In [ ]:
%pip install pytorch_pretrained_bert --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 4.3 MB/s 


# T5 for summarization

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

# def ppp():
#   arr = textwrap.wrap(paragraph, 4000)
#   summary = ''
#   for i in arr:
#     summary += pipe( i ) [0]["summary_text"].replace(" .", ".\n")
#   return summary

def generate_summary_t5(text):
    preprocess_text = text.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    # summmarize
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=100,
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return output

In [ ]:
?pipe

In [ ]:
t5_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary_t5(df['content'][i] ) 
  t5_summaries.append( summary )

df['t5'] = t5_summaries

100%|██████████| 13/13 [04:19<00:00, 19.94s/it]


In [ ]:
df

,content,t5
0,Welcome back to Pattern Recognition!\nToday we...,we have a set of new vectors that are lambda a...
1,Welcome back to Pattern Recognition!\nToday we...,the laplacian support vector machine is essent...
2,Welcome back to pattern recognition.\nToday we...,this is the longest projection of our negative...
3,Welcome back to pattern recognition.\nToday we...,we want to introduce the kernel PCA and look i...
4,"The Chrysler Building, the famous art deco New...",the building sold fairly quickly after being p...
5,The most sophisticated form of HIE creates sem...,the most sophisticated form of HIE creates sem...
6,Welcome back to pattern recognition.\nToday we...,we want to talk about optimization and of cour...
7,dr karen bell is the chair of the certificatio...,dr karen bell is the president of the national...
8,Dr. Karen Bell is the chair of the Certificat...,health informatics is a new era of information...
9,"All right, so here is a rule we're going to c...",if we repeat this update rule over the finite ...


In [ ]:
%cd ..
df.to_csv('summaries_t5.csv', index=False)

/content
